In [1]:
import os

from dotenv import load_dotenv
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPEN_AI_KEY')

In [3]:
%pip install youtube-transcript-api
%pip install pytube

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### 1. Simple Videos

In [2]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=54_xZYjWzWE", add_video_info=True)

AttributeError: type object 'YoutubeLoader' has no attribute 'from_youtube_url'

In [3]:
result = loader.load()

In [4]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>
Found video from Saturday Night Live that is 224 seconds long

[Document(page_content='LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDRO

In [12]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [9]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

' Pedro Pascal shared his experience shooting for HBO\'s "The Last of Us" and his journey to becoming a recognizable actor. He also spoke about his Chilean family, who were initially not proud of his acting career, but are now supportive. He concluded his speech by introducing Coldplay, who will be performing at the show.'

### 2. Long Videos

In [6]:
#loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=54_xZYjWzWE", add_video_info=True)
loader = YoutubeLoader("UeLLmJEKVps", True)

In [7]:
result = loader.load()

In [23]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 15720 tokens (15464 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [10]:
texts = text_splitter.split_documents(result)

In [13]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"[Music] hello everyone welcome back for this new episode of coding design system and this episode is the last one of the season so um we're gonna wrap it after this one and we will meet again in august and to conclude this season i'm i'm more than happy to have bradfrost on the show with me hey brad who are you hi good thanks so much for having me this is an honor thanks for accepting the invitation it's a it's an honor for me to to have you on stream with me um it's really great so um let me i don't know introduce yourself you're more than famous for a lot of things inventing a lot of concept in the past and we we're gonna discuss about it uh um in a bit but um you want to add some few words about what you're doing right now and and the project you are working on things like that and that's it that's a big question yeah i'm uh i'm a design system consultant front-end

' Brad Frost, inventor of the concept of atomic design, joins the show to discuss his work as a design system consultant. Atomic design is a powerful concept for creating a feedback loop between design systems components and the products they serve. Design systems are a tool to bridge the gap between designers and developers, but often there is tension between the two groups. Design system consultants work to show how designers and developers can work together to improve the design system.'

### 3. Multiple Videos

In [32]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [33]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' This article discusses the process of building a backyard golf green, including the cost of materials and the steps involved. Bella Ramsey and Pedro Pascal also discuss their experiences on the set of Game of Thrones and The Last of Us, including their first impressions of each other, what they took from set, and what item from the Game of Thrones world they would bring to The Last of Us. Bella talks about her most prized possession, a tear stick, and her best attribute in a survival scenario. Finally, they discuss their first jobs.'